In [1]:
from rapidfuzz import fuzz

SIMILARITY_THRESHOLD = 0.8   
USE_PREPROCESSING = True     


In [3]:
import string

def preprocess(name: str) -> str:
    """
    Простая предобработка названия товара:
    - приводим к нижнему регистру;
    - убираем пунктуацию;
    - заменяем несколько пробелов одним.
    """
    text = name.lower()
    # убираем знаки пунктуации
    translator = str.maketrans("", "", string.punctuation)
    text = text.translate(translator)
    # нормализуем пробелы
    words = text.split()
    return " ".join(words)


In [4]:
from typing import Dict, Tuple

def load_items(path: str) -> Dict[str, str]:
    """
    Читает файл формата:
    ID\tНазвание товара
    и возвращает словарь {id: name}.
    """
    items = {}
    with open(path, encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split("\t", maxsplit=1)
            if len(parts) != 2:
                # пропускаем некорректные строки
                continue
            item_id, name = parts
            items[item_id] = name
    return items


In [5]:
def similarity_score(name1: str, name2: str) -> float:
    """
    Возвращает коэффициент схожести двух строк от 0 до 1.
    Используется token_sort_ratio из rapidfuzz. [web:11][web:23]
    """
    if USE_PREPROCESSING:
        name1 = preprocess(name1)
        name2 = preprocess(name2)
    # token_sort_ratio возвращает значение 0..100,
    # поэтому делим на 100, чтобы получить 0..1
    score = fuzz.token_sort_ratio(name1, name2)
    return score / 100.0


In [6]:
def find_duplicates(catalog: Dict[str, str],
                    new_items: Dict[str, str],
                    threshold: float) -> Dict[str, list]:
    """
    Для каждого нового товара ищет потенциальные дубликаты в каталоге.
    Возвращает словарь:
    { new_id: [ { "catalog_id": ..., "similarity_score": ... }, ... ] }
    """
    result = {}

    for new_id, new_name in new_items.items():
        duplicates = []
        for cat_id, cat_name in catalog.items():
            score = similarity_score(new_name, cat_name)
            if score >= threshold:
                duplicates.append({
                    "catalog_id": cat_id,
                    "similarity_score": round(score, 2)  # округлим до двух знаков
                })

        # Можно отсортировать по убыванию схожести, чтобы сначала были самые близкие
        duplicates.sort(key=lambda x: x["similarity_score"], reverse=True)
        result[new_id] = duplicates

    return result


In [7]:
import json

# Пути к файлам
catalog_path = "catalog.txt"
new_items_path = "new_items.txt"

catalog_items = load_items(catalog_path)
new_items = load_items(new_items_path)

duplicates = find_duplicates(catalog_items, new_items, SIMILARITY_THRESHOLD)

# Сохраняем результат в файл duplicates.json
with open("duplicates.json", "w", encoding="utf-8") as f:
    json.dump(duplicates, f, ensure_ascii=False, indent=4)

duplicates  # вывод для проверки в ноутбуке


{'2001': [{'catalog_id': '1001', 'similarity_score': 0.86}],
 '2002': [],
 '2003': [{'catalog_id': '1003', 'similarity_score': 0.84}],
 '2004': [],
 '2005': []}